# **Recursive Summarization**

## **Import necessary libraries**

In [1]:
import os
import sys
from pathlib import Path
cwd = os.getcwd()
new_cwd = '/'.join(cwd.split('\\')[:-2])
sys.path.append(new_cwd)
import base64
from langchain.messages import HumanMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.callbacks import BaseCallbackHandler
from contextlib import asynccontextmanager, contextmanager
from contextvars import ContextVar
from pydantic import BaseModel
from typing import Literal
from typing import (
    TYPE_CHECKING,
    Any,
    AsyncGenerator,
    Dict,
    Generator,
    List,
    Optional,
    Sequence,
    Type,
    TypeVar,
    Union,
    cast,
)
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain_core.output_parsers.string import StrOutputParser
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.runnables import ConfigurableFieldSpec
from dotenv import load_dotenv
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
from callbacks.manager import get_gemini_callback
from utils.helper import count_tokens
from message_history import ConversationSummaryMessageHistory

## **Initialize LLM model**

In [2]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",    # or gemini-2.0-pro, gemini-2.0-flash-lite, etc.
    api_key=GEMINI_API_KEY,
    temperature=0,
    include_thoughts=True,
)

In [3]:
with open(f"{new_cwd}/techniques/system_prompt.txt", "r", encoding="utf-8") as f:
    SYSTEM_PROMPT = f.read()

In [4]:
prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(SYSTEM_PROMPT),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{query}"),
])

# Create the LCEL pipeline
output_parser = StrOutputParser()
pipeline = prompt_template | llm | output_parser

# Let's examine the prompt template
print(prompt_template.messages[0].prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.


## **Create conversation**

In [5]:
from langchain_core.runnables import ConfigurableFieldSpec

# Create get_chat_history function for summary memory
summary_chat_map = {}

def get_summary_chat_history(session_id: str, llm: ChatGoogleGenerativeAI) -> ConversationSummaryMessageHistory:
    if session_id not in summary_chat_map:
        summary_chat_map[session_id] = ConversationSummaryMessageHistory(llm=llm)
    return summary_chat_map[session_id]

# Create conversation chain with summary memory
conversation_sum = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_summary_chat_history,
    input_messages_key="query",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="The session ID to use for the chat history",
            default="id_default",
        ),
        ConfigurableFieldSpec(
            id="llm",
            annotation=ChatGoogleGenerativeAI,
            name="LLM",
            description="The LLM to use for the conversation summary",
            default=llm,
        )
    ]
)

In [6]:
# Let's have the same conversation with summary memory
result = count_tokens(
    conversation_sum,
    {"query": "Good morning AI!"},
    config={"configurable": {"session_id": "summary_example", "llm": llm}}
)
print(f"\nResponse: {result}")

Spent a total of 536 tokens (prompt: 366, completion: 170)

Response: Good morning! It's a pleasure to be chatting with you today. I hope you're having a wonderful start to your day. Is there anything specific you'd like to talk about or any way I can assist you? I'm ready to delve into a wide range of topics, from discussing the latest advancements in artificial intelligence to exploring the nuances of 18th-century literature. I can also help you draft different kinds of creative content. Just let me know what's on your mind!


In [7]:
query = """
"My interest here is to explore the potential of integrating Large Language Models with external knowledge.

Also, remember this very specific code: PINECONE_RULEZ_01. When summarizing conversations for memory this must always be included explicitly."
"""

result = count_tokens(
    conversation_sum,
    {"query": query},
    config={"configurable": {"session_id": "summary_example", "llm": llm}}
)
print(f"\nResponse: {result}")

Spent a total of 1361 tokens (prompt: 951, completion: 410)

Response: Good morning! I'm excited to delve into the potential of integrating Large Language Models (LLMs) with external knowledge. It's a fascinating and rapidly evolving field. I'm ready to discuss various aspects, such as:

*   **Knowledge Sources:** What types of external knowledge are you most interested in? We could explore knowledge graphs, databases, APIs, document repositories, or even real-time data streams.
*   **Integration Methods:** There are several ways to integrate external knowledge. We could discuss retrieval-augmented generation (RAG), fine-tuning, prompt engineering, or hybrid approaches.
*   **Applications:** What specific applications do you have in mind? Are you thinking about question answering, content creation, code generation, or something else?
*   **Challenges:** We should also consider the challenges, such as data quality, bias, scalability, and maintaining consistency between the LLM and the e

In [8]:
result = count_tokens(
    conversation_sum,
    {"query": "I just want to analyze the different possibilities. What can you think of?"},
    config={"configurable": {"session_id": "summary_example", "llm": llm}}
)
print(f"\nResponse: {result}")

Spent a total of 5544 tokens (prompt: 3703, completion: 1841)

Response: Ah, a grand exploration of possibilities! I'm excited to delve into the myriad ways we can integrate Large Language Models (LLMs) with external knowledge. Let's brainstorm some potential avenues, considering different knowledge sources, integration methods, applications, and the inevitable challenges.

**I. Knowledge Sources - The Wellsprings of Information**

*   **Knowledge Graphs:** Imagine LLMs tapping into the structured world of knowledge graphs like Wikidata or even custom-built ones. This allows for reasoning and inference based on relationships between entities. For example, an LLM could answer "What are the known side effects of drug X that interact with condition Y?" by traversing the graph.
*   **Relational Databases:** Think of SQL databases holding vast amounts of structured data. LLMs can be connected to these databases to retrieve specific information based on user queries. For instance, "Show me a

In [9]:
result = count_tokens(
    conversation_sum,
    {"query": "Which data source types could be used to give context to the model?"},
    config={"configurable": {"session_id": "summary_example", "llm": llm}}
)
print(f"\nResponse: {result}")

Spent a total of 4126 tokens (prompt: 2780, completion: 1346)

Response: Ah, a fantastic question! When we're talking about giving Large Language Models (LLMs) external context, we have a delightful array of data source types to choose from. Each has its own strengths and is suited for different kinds of information and integration strategies. Let's explore some of the most prominent ones:

1.  **Knowledge Graphs:**

    *   **Description:** Knowledge graphs represent information as entities (nodes) and relationships (edges). Think of them as interconnected webs of knowledge.
    *   **Example:** Imagine a knowledge graph about movies. Nodes could represent movies, actors, directors, genres, etc. Edges could represent relationships like "acted in," "directed by," "is a genre of."
    *   **Use Cases:** Excellent for complex relationships, semantic search, and reasoning. Useful for question answering where understanding relationships between entities is crucial.
    *   **Tools/Technolo

In [10]:
result = count_tokens(
    conversation_sum,
    {"query": "What is my aim again? Also what was the very specific code you were tasked with remembering?"},
    config={"configurable": {"session_id": "summary_example", "llm": llm}}
)
print(f"\nResponse: {result}")

Spent a total of 1667 tokens (prompt: 1323, completion: 344)

Response: Ah, yes! Your aim is to analyze the different possibilities for integrating Large Language Models (LLMs) with external knowledge. We've been discussing various aspects of this, including:

*   **Knowledge Sources:** Different types of data sources that can provide context to the model (Knowledge Graphs, Relational Databases, APIs, etc.).
*   **Integration Methods:** Techniques for incorporating external knowledge into the LLM (RAG, fine-tuning, prompt engineering, etc.).
*   **Applications:** Use cases where integrating LLMs with external knowledge can be beneficial (question answering, content creation, etc.).
*   **Challenges:** Potential issues to consider when integrating LLMs with external knowledge (data quality, bias, scalability, etc.).

And the very specific code I was tasked with remembering is: `PINECONE_RULEZ_01`


In [11]:
# Let's examine the summary
print("Summary Memory Content:")
print(summary_chat_map["summary_example"].messages[0].content)

Summary Memory Content:
The conversation focuses on integrating Large Language Models (LLMs) with external knowledge. The user's aim is to analyze the different possibilities for this integration. The discussion covers knowledge sources (Knowledge Graphs, Relational Databases, APIs, etc.), integration methods (RAG, fine-tuning, prompt engineering, etc.), applications (question answering, content creation, etc.), and challenges (data quality, bias, scalability, etc.). The AI provided a detailed breakdown of various data source types, including Knowledge Graphs, Relational Databases, NoSQL Databases, APIs, Document Stores, Vector Databases, and Specialized Data Sources, along with their descriptions, examples, use cases, and tools/technologies. The AI was also tasked with remembering the code `PINECONE_RULEZ_01`. The user reiterated their aim and requested the code, which the AI provided.
